In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import time

# Global variables to store story state
story_history = []
current_story = ""
story_choices = []
current_input = ""

def initialize_story(user_input):
    """Initialize the story with user input and generate first response"""
    global current_story, story_choices, story_history, current_input
    
    if not user_input.strip():
        # hide everything if empty
        return (
            gr.update(visible=False, value=""),  # chat_output
            gr.update(visible=False),  # story_image
            gr.update(visible=False),  # audio_btn
            gr.update(visible=False),  # choice1
            gr.update(visible=False),  # choice2
            gr.update(visible=False),  # choice3
            gr.update(visible=False),  # choice4
            gr.update(visible=False),  # prev_btn
            gr.update(visible=False),  # next_btn
            gr.update(visible=True, value=""),  # keep user_input visible
            gr.update(visible=True)  # keep submit_btn visible
        )
    
    # Simulate story generation
    story_response = f"Once upon a time, {user_input}. The adventure begins as mysterious events unfold around our protagonist..."
    
    # Generate story choices
    choices = [
        "Investigate the mysterious sound",
        "Seek help from a friend", 
        "Explore the hidden passage",
        "Wait and observe carefully"
    ]
    
    # Update global state
    current_story = story_response
    story_choices = choices
    story_history = [{"input": user_input, "output": story_response}]
    current_input = user_input
    
    # Truncate input for display
    display_input = user_input if len(user_input) <= 50 else user_input[:50] + "..."
    
    # Return updates, hiding both the original input box AND the submit button
    return (
        gr.update(visible=True, value=story_response),  # chat_output
        gr.update(visible=True),  # story_image
        gr.update(visible=True),  # audio_btn
        gr.update(visible=True, value=choices[0]),  # choice1
        gr.update(visible=True, value=choices[1]),  # choice2
        gr.update(visible=True, value=choices[2]),  # choice3
        gr.update(visible=True, value=choices[3]),  # choice4
        gr.update(visible=True),  # prev_btn
        gr.update(visible=True),  # next_btn
        gr.update(visible=False),  # hide user_input
        gr.update(visible=False)   # hide submit_btn
    )


def play_audio():
    """Play the actual audio file"""
    return "/content/wizard_spell.wav"


def make_choice(choice_num):
    """Handle story choice selection"""
    global current_story, story_choices, story_history, current_input
    
    if choice_num < len(story_choices):
        selected_choice = story_choices[choice_num]
        current_input = selected_choice
        
        # Simulate story continuation based on choice
        continuations = [
            f"You decided to {selected_choice.lower()}. As you move forward, you discover something unexpected that changes everything...",
            f"Following your instinct to {selected_choice.lower()}, you find yourself in a completely new situation filled with possibilities...",
            f"Your choice to {selected_choice.lower()} leads to an surprising encounter that will shape the rest of your journey...",
            f"By choosing to {selected_choice.lower()}, you unlock a new chapter in your adventure that you never saw coming..."
        ]
        new_story = continuations[choice_num % len(continuations)]
        current_story = new_story
        
        # Generate new choices
        new_choices = [
            f"Continue exploring {choice_num + 1}A",
            f"Take a different path {choice_num + 1}B", 
            f"Seek more information {choice_num + 1}C",
            f"Make a bold decision {choice_num + 1}D"
        ]
        story_choices = new_choices
        story_history.append({"input": selected_choice, "output": new_story})
        
        # Update input display
        display_input = selected_choice if len(selected_choice) <= 50 else selected_choice[:50] + "..."
        
        return (
            gr.update(value=new_story),      # chat_output
            gr.update(value=new_choices[0]),  # choice1
            gr.update(value=new_choices[1]),  # choice2
            gr.update(value=new_choices[2]),  # choice3
            gr.update(value=new_choices[3])   # choice4
        )    
    return tuple([gr.update()]*5)


def go_previous():
    return "📖 Navigated to previous story segment..."

def go_next():
    return "📖 Navigated to next story segment..."

# Create the Gradio interface
with gr.Blocks(title="Story Generation Platform", theme=gr.themes.Soft(), css="""
    .audio-button { 
        width: 100px !important; 
        height: 100px !important; 
        border-radius: 50% !important; 
        min-width: 100px !important; 
        font-size: 30px !important;
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
        border: 3px solid #ffffff !important;
        box-shadow: 0 8px 32px rgba(102, 126, 234, 0.4) !important;
        transition: all 0.3s ease !important;
    }
    .audio-button:hover {
        transform: scale(1.05) !important;
        box-shadow: 0 12px 40px rgba(102, 126, 234, 0.6) !important;
    }
    .input-small { max-height: 45px !important; }
    .image-center { display: flex !important; justify-content: center !important; align-items: center !important; }
    .enter-button { margin-top: 20px !important; }
""") as demo:
    gr.Markdown("# 📚 Story Generation Platform")
    gr.Markdown("Enter your story idea to begin an interactive adventure!")
    with gr.Row():
        with gr.Column(scale=1):
            audio_btn = gr.Button("🔊", visible=False, variant="primary", elem_classes="audio-button")
        with gr.Column(scale=2, elem_classes="image-center"):
            story_image = gr.Image(value="/content/complex_test_image_4.png", visible=False, height=300, width=400, label="Story Scene")
        with gr.Column(scale=1): pass
    user_input = gr.Textbox(placeholder="Enter your story beginning...", label="Story Input", lines=2)
    submit_btn = gr.Button("Enter", variant="primary", elem_classes="enter-button")

    chat_output = gr.Textbox(label="Story Output", lines=6, interactive=False, visible=False)
    audio_output = gr.Audio(visible=False, autoplay=True)
    gr.Markdown("### Story Choices", visible=False, elem_id="choices-header")
    with gr.Row():
        choice1_btn = gr.Button("Choice 1", visible=False, variant="secondary")
        choice2_btn = gr.Button("Choice 2", visible=False, variant="secondary")
        choice3_btn = gr.Button("Choice 3", visible=False, variant="secondary")
        choice4_btn = gr.Button("Choice 4", visible=False, variant="secondary")
    with gr.Row():
        prev_btn = gr.Button("⬅️ Previous", visible=False, variant="outline")
        gr.HTML("<div style='flex-grow: 1;'></div>")
        next_btn = gr.Button("Next ➡️", visible=False, variant="outline")
    
    # Event handlers
    submit_btn.click(
        fn=initialize_story,
        inputs=[user_input],
        outputs=[
            chat_output, story_image, audio_btn,
            choice1_btn, choice2_btn, choice3_btn, choice4_btn,
            prev_btn, next_btn, user_input, submit_btn
        ]
    )
    audio_btn.click(fn=play_audio, outputs=[audio_output])
    choice1_btn.click(fn=lambda: make_choice(0), outputs=[chat_output, choice1_btn, choice2_btn, choice3_btn, choice4_btn])
    choice2_btn.click(fn=lambda: make_choice(1), outputs=[chat_output, choice1_btn, choice2_btn, choice3_btn, choice4_btn])
    choice3_btn.click(fn=lambda: make_choice(2), outputs=[chat_output, choice1_btn, choice2_btn, choice3_btn, choice4_btn])
    choice4_btn.click(fn=lambda: make_choice(3), outputs=[chat_output, choice1_btn, choice2_btn, choice3_btn, choice4_btn])
    prev_btn.click(fn=go_previous, outputs=[chat_output])
    next_btn.click(fn=go_next, outputs=[chat_output])

if __name__ == "__main__":
    demo.launch(share=True, debug=True)